In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense

In [2]:
# Load historical stock price data
# For this example, I'll use the historical stock prices of Apple Inc. (AAPL)

In [3]:
np.random.seed(42)
stock_data = pd.DataFrame({'Close': np.random.rand(2520) * 100}, index=pd.date_range('2010-01-01', periods=2520, freq='B'))


# Normalize the data

In [4]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(stock_data['Close'].values.reshape(-1, 1))


# Prepare the data for LSTM

In [5]:
def create_dataset(dataset, time_steps=1):
    dataX, dataY = [], []
    for i in range(len(dataset) - time_steps):
        a = dataset[i:(i + time_steps), 0]
        dataX.append(a)
        dataY.append(dataset[i + time_steps, 0])
    return np.array(dataX), np.array(dataY)

time_steps = 60  # You can experiment with different time steps
X, y = create_dataset(scaled_data, time_steps)


# Reshape input data to fit the LSTM model (samples, time steps, features)

In [6]:
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

# Build the LSTM model

In [7]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], 1)))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dense(units=25))
model.add(Dense(units=1))

model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model

In [8]:
model.fit(X, y, epochs=10, batch_size=32)

Epoch 1/10
77/77 [==============================] - 10s 56ms/step - loss: 0.0972
Epoch 2/10
77/77 [==============================] - 4s 55ms/step - loss: 0.0867
Epoch 3/10
77/77 [==============================] - 4s 55ms/step - loss: 0.0872
Epoch 4/10
77/77 [==============================] - 4s 55ms/step - loss: 0.0861
Epoch 5/10
77/77 [==============================] - 4s 57ms/step - loss: 0.0859
Epoch 6/10
77/77 [==============================] - 4s 56ms/step - loss: 0.0864
Epoch 7/10
77/77 [==============================] - 4s 56ms/step - loss: 0.0853
Epoch 8/10
77/77 [==============================] - 4s 57ms/step - loss: 0.0861
Epoch 9/10
77/77 [==============================] - 4s 56ms/step - loss: 0.0857
Epoch 10/10
77/77 [==============================] - 4s 57ms/step - loss: 0.0855


# Make predictions
# For simplicity, let's predict the next 10 days

In [10]:
future_days = 10
last_days = scaled_data[-time_steps:]
last_days = np.reshape(last_days, (1, time_steps, 1))

predicted_prices = []
for i in range(future_days):
    predicted_price = model.predict(last_days)
    last_days = np.reshape(last_days, (last_days.shape[0], last_days.shape[1]))  # Flatten the array
    last_days = np.append(last_days[0, 1:], predicted_price)  # Remove the first element and append the predicted value
    last_days = np.reshape(last_days, (1, time_steps, 1))  # Reshape for the next prediction
    predicted_prices.append(predicted_price)

1/1 [==============================] - 0s 26ms/step


# Inverse transform the predicted prices to get actual stock prices

In [11]:
predicted_prices = np.array(predicted_prices).reshape(-1, 1)
predicted_prices = scaler.inverse_transform(predicted_prices)

# Display the predicted prices

In [12]:
print(predicted_prices)

[[58.67731 ]
 [58.73767 ]
 [58.770565]
 [58.792774]
 [58.810444]
 [58.82643 ]
 [58.84184 ]
 [58.856976]
 [58.871742]
 [58.88597 ]]


In [13]:
# This project has done here. Now we are able to predict price of Apple Inc. (AAPL) company.
# Thank you